## IMPORT LIBRARIES

In [31]:
# Import necessary libraries
import tkinter as tk
from tkinter import simpledialog, filedialog
import pandas as pd
from pyDOE2 import fullfact
import os
import json
import datetime
import sys
from IPython import get_ipython
import numpy as np
import openpyxl
from openpyxl import Workbook


## CREATE FACTORIAL DESIGN PLAN

In [32]:
import pandas as pd
from pyDOE2 import fullfact
import tkinter as tk
from tkinter import filedialog
import os

def generate_factor_levels_df(factors, levels, high_levels, low_levels):
    """
    Generate a DataFrame with factor names, number of levels, and high and low levels.

    Args:
    - factors (list): List of factor names.
    - levels (list): List containing the number of levels for each factor.
    - high_levels (list): List of high factor levels for each factor.
    - low_levels (list): List of low factor levels for each factor.

    Returns:
    - df (DataFrame): DataFrame representing factor names, levels, high, mid, and low levels.
    """
    # Check if the number of factors matches the number of levels
    if len(factors) != len(levels) != len(high_levels) != len(low_levels):
        raise ValueError("Number of factors, levels, high, and low levels must be the same.")
    # Check if all levels are positive integers
    if not all(isinstance(level, int) and level > 0 for level in levels):
        raise ValueError("Levels must be positive integers.")

    # Create lists to hold factor names, levels, high, mid, and low levels
    factor_list = []
    level_list = []
    high_list = []
    mid_list = []
    low_list = []

    # Iterate through factors and calculate mid levels if levels > 2
    for factor, num_levels, high_level, low_level in zip(factors, levels, high_levels, low_levels):
        factor_list.append(factor)
        level_list.append(num_levels)
        high_list.append(high_level)
        low_list.append(low_level)

        # Calculate mid levels
        if num_levels > 2:
            mid = [(low_level + ((high_level - low_level) * i) / (num_levels - 1)) for i in range(1, num_levels - 1)]
            # Round mid levels to two decimal places
            mid = sorted([round(m, 1) for m in mid], reverse=True)
            mid_list.append(mid)
        else:
            mid_list.append(None)

    # Create a dictionary to hold factor names, levels, high, mid, and low levels
    data = {'Factor': factor_list, 'Levels': level_list, 'High(+1)': high_list, '-1<Mid<1': mid_list, 'Low(-1)': low_list}
    # Convert the dictionary into a DataFrame
    df = pd.DataFrame(data)

    return df

def create_full_factorial_design(factor_levels_df):
    """
    Generate a full factorial design DataFrame based on specified factors and levels.

    Args:
    - factor_levels_df (DataFrame): DataFrame containing factor names, levels, highs, mids, and lows.

    Returns:
    - df (DataFrame): DataFrame representing the full factorial design.
    """
    # Extract factor names and levels from the DataFrame
    factors = factor_levels_df['Factor'].tolist()
    levels = factor_levels_df['Levels'].tolist()
    highs = factor_levels_df['High(+1)'].tolist()
    mids = factor_levels_df['-1<Mid<1'].tolist()
    lows = factor_levels_df['Low(-1)'].tolist()

    # Generate the full factorial design using pyDOE2's fullfact() function
    design = fullfact(levels)

    # Convert the design into a DataFrame with appropriate column names
    df = pd.DataFrame(design, columns=factors)

    # Create new columns for highs, lows, and mids
    for factor, high, mid, low in zip(factors, highs, mids, lows):
        df[f'{factor}_high'] = high
        df[f'{factor}_mid'] = mid
        df[f'{factor}_low'] = low

    # Apply custom mapping to each value in the DataFrame and add mapped columns
    for column, high, low, mid in zip(df.columns, highs, lows, mids):
        num_levels = factor_levels_df.loc[factor_levels_df['Factor'] == column, 'Levels'].iloc[0]
        mapped_values = [custom_mapping(value, num_levels, high, low, mid) for value in df[column]]
        df[column + '_mapped'] = mapped_values

    return df

def custom_mapping(value, num_levels, high, low, mid):
    """
    Custom mapping function to transform values according to specific rules.

    Args:
    - value (int): Value representing factor level.
    - num_levels (int): Number of levels for the factor.
    - high (float): High factor levels.
    - low (float): Low factor levels.
    - mid (float): Mid factor levels.

    Returns:
    - mapped_value (float): Transformed value.
    """
    if value < 0 or value >= num_levels:
        return value  # Value outside the defined levels, return as is

    # Calculate the spacing between each level
    spacing = 2 / (num_levels - 1)

    # Map the value to the corresponding value in the range [-1, 1]
    mapped_value = -1 + value * spacing

    return mapped_value

def get_user_input():
    """
    Prompt the user to input factors, levels, number of replicates, Excel file name, and export folder.
    Additionally, prompt for high, mid, and low factor levels for each factor based on the number of levels.

    Returns:
    - factors (list): List of factor names.
    - levels (list): List of levels corresponding to each factor.
    - high_levels (list): List of high factor levels for each factor.
    - low_levels (list): List of low factor levels for each factor.
    - num_replicates (int): Number of replicates.
    - excel_file_name (str): Excel file name.
    - export_folder (str): Export folder path.
    """
    root = tk.Tk()
    root.withdraw()  # Hide the root window

    # Temporary default values for testing
    factors = ['A', 'B', 'C']
    levels = [4, 3, 2]
    high_levels = [100, 50, 300]
    low_levels = [20, 10, 280]
    num_replicates = 1
    excel_file_name = 'Test'

    # Prompt user to select the export folder
    export_folder = filedialog.askdirectory(title="Select Export Folder")

    return factors, levels, high_levels, low_levels, num_replicates, excel_file_name, export_folder

def main():
    try:
        # Get user input
        factors, levels, high_levels, low_levels, num_replicates, excel_file_name, export_folder = get_user_input()

        # Generate factor levels DataFrame
        factor_levels_df = generate_factor_levels_df(factors, levels, high_levels, low_levels)

        # Create full factorial design DataFrame
        full_factorial_df = create_full_factorial_design(factor_levels_df)

        # Duplicate the full factorial design DataFrame
        full_factorial_df_duplicated = pd.concat([full_factorial_df] * num_replicates, ignore_index=True)

        # Append '.xlsx' extension if not provided
        if not excel_file_name.endswith('.xlsx'):
            excel_file_name += '.xlsx'

        # Write the DataFrame to an Excel file in the export folder
        excel_file_path = os.path.join(export_folder, excel_file_name)
        full_factorial_df_duplicated.to_excel(excel_file_path, index=False)
        print("Excel file saved successfully.")

    except Exception as e:
        print("An error occurred:", e)

if __name__ == "__main__":
    main()


An error occurred: Length of values (2) does not match length of index (24)
